In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix as cm
from mlxtend.evaluate import BootstrapOutOfBag
import matplotlib.pyplot as plt
import pandas as pd
import time
import random
import numpy

from numpy import *
from sklearn import *
from pandas import *
from scipy import stats

xl = pd.ExcelFile('data2.xlsx')
xl.sheet_names # we'll take 7th
dfs = {sheet: xl.parse(sheet) for sheet in xl.sheet_names}
data1 = dfs['7']
data2 = dfs['1'].loc[:,['Patient','Age at Diagnosis']].drop([554]).drop_duplicates()
# import datas/et1
data3 = pd.read_csv('data1.csv')

combined_data = data1.set_index('Patient').join(data2.set_index('Patient')).join(data3.set_index('Patient'))

combined_data['label'] = (combined_data['Patient Type'] == 'Healthy').astype(int)
combined_data = combined_data.drop(['Patient Type'],axis=1)
print('The number of samples and features are %d and %d, respectively'%(combined_data.shape[0],combined_data.shape[1]))


x = combined_data.iloc[:, 0:44]
x[isnan(x)] = 0
y=combined_data.iloc[:,44]

In [ ]:
 def Score_632plus(clf,x,y,x_train,y_train,x_test,y_test): 
    y_score_test = clf.predict_proba(x_test)[:,1]
    y_score_train = clf.predict_proba(x_train)[:,1]  
    
    # Compute error based on auc
    E_resub =  metrics.brier_score_loss(y_train, y_score_train)
    E_loop = metrics.brier_score_loss(y_test, y_score_test)
    # compute no-information-rate
    y_pred = clf.predict(x)
    length = len(y_pred)
    gamma = sum(y==0)/length*(1-sum(y_pred==0)/length)+sum(y==1)/length*(1-sum(y_pred==1)/length)
    
    E_632 = 0.368*E_resub + 0.632*E_loop
    
    # compute R
    if gamma > E_resub:
        R = (E_loop - E_resub) / (gamma - E_resub)
    else:
        R = 0
        E_resub = gamma
    
    # compute weight and adjusted Score
    E_632plus = E_632 + (E_resub - E_loop)*(0.368*0.632*R/(1-0.368*R))
    return E_632plus

In [ ]:
def SFLA_SVM(x_train, y_train,x_traincv, y_traincv,x_testcv, y_testcv,kernel,C,gamma=False,degree=False,coef0=False):
    clf = svm.SVC(C=C, kernel=kernel,gamma=gamma, coef0=coef0, probability=True,random_state=920).fit(x_traincv, y_traincv)
    bscore = -Score_632plus(clf,x_train, y_train,x_traincv, y_traincv,x_testcv, y_testcv)
    return bscore

In [ ]:
def SFLA_SVM_CV(x_train, y_train,n,kernel,C,gamma=False,degree=False,coef0=False):
    '''
    n: number of splits for k-fold
    
    '''
    KF = KFold(n_splits=n,shuffle=True, random_state=920)
    f = []
    for train_indexcv,test_indexcv in KF.split(x_train):
        x_traincv, x_testcv = x_train.iloc[train_indexcv][:], x_train.iloc[test_indexcv][:]
        y_traincv, y_testcv = y_train.iloc[train_indexcv][:], y_train.iloc[test_indexcv][:]
        fq = SFLA_SVM(x_train, y_train,x_traincv, y_traincv,x_testcv, y_testcv,kernel,C,gamma) 
        f.append(fq) 
    f = mean(f)
    return f

In [ ]:
def SFLA_RBF(num_parameter,num_global,num_local,m,n,q,n1,kernel,rangeC,rangeGamma,x_train,y_train):
    '''
    num_parameter: int, number of parameter to optimize
    
    num_global: int, the maximum number of global iterations
    
    num_local: int, the maximum number of local iterations
    
    m : int, the number of memeplexes
    
    n : int, the number of frogs in each memeplex
    
    q : int, the number of frogs in submemeplex
    
    n1:  number of splits for cross validation for inner loop
    
    rangeC: list, float, range of parameter C,eg.[10**-2, 10**2]
    
    rangeGamma: list, float, range of parameter Gamma,eg.[10**-6, 1]

    x_train: feature

    y_train: lable

    '''

    #--- Step 0--Initialize parameters ---#
    sizeC = 2
    sizeGamma = 2
    max_step =  [(rangeC[1]-rangeC[0])/sizeC,(rangeGamma[1]-rangeGamma[0])/sizeGamma]# maximum step size
    
    #--- Step 1--Generate initial population ---#
    frogC = 10**random.uniform(log10(rangeC[0]),log10(rangeC[1]),m*n)
    frogGamma = 10**random.uniform(log10(rangeGamma[0]),log10(rangeGamma[1]),m*n)
    frog = c_[frogC,frogGamma]

    # Compute the performance value for each frog on validation data #
    KF = KFold(n_splits=n1,shuffle=True, random_state=920)
    f = zeros((m*n,n1))
    j = 0
    for train_indexcv,test_indexcv in KF.split(x_train):
        x_traincv, x_testcv = x_train.iloc[train_indexcv][:], x_train.iloc[test_indexcv][:]
        y_traincv, y_testcv = y_train.iloc[train_indexcv][:], y_train.iloc[test_indexcv][:]
        for i in range(m*n):
            f[i,j] = SFLA_SVM(x_train, y_train,x_traincv, y_traincv,x_testcv, y_testcv,kernel,frog[i,0],frog[i,1])
        j+=1
    f = f.mean(axis=1)
    f_parameter = c_[f,frog]

    #--- Step 2--Rank frogs ---#
    f_parameter = f_parameter[argsort(f_parameter[:,0])[::-1]]


    #######--- Global search start---######
    i_global = 0
    flag = 0
    fBest_iteration = f_parameter[0,0]
    weights = [2*(n+1-j)/(n*(n+1)) for j in range(1,n+1)] # weights of ranked frogs in each memeplex
    while i_global < num_global:
        frog_gb = f_parameter[0,0] # mark the global best frog      
        #--- Step 3--Partition frogs into memeplexes ---#
        memeplexes = zeros((m,n,num_parameter+1)) # [memeplexes, frog in memeplex,[f,C,Gamma] ]
        for i in range(m):
            memeplexes[i] = f_parameter[linspace(i,m*n+i,num=n,endpoint=False,dtype=int)]

        #######--- Local search start---######
        #--- Step 4--Memetic evolution within each memeplex ---#
        im = 0 # the number of memeplexes that have been optimized
        while im < m:
            i_local = 0 # counts the number of local evolutionary steps in each memeplex
            while i_local < num_local:

                #--- Construct a submemeplex ---#
                rValue = random.random(n)*weights # random value with probability weights
                subindex = sort(argsort(rValue)[::-1][0:q]) # index of selected frogs in memeplex 
                submemeplex = memeplexes[im][subindex] # form submemeplex

                #--- Improve the worst frog's position ---#
                # Learn from local best Pb #
                Pb = submemeplex[0] # mark the best frog in submemeplex
                Pw = submemeplex[q-1] # mark the worst frog in memeplex
                S = (Pb-Pw)[1:]*(Pb-Pw)[0] 
                Uq = Pw[1:]+S
                # Check feasible space and the performance #
                if (rangeC[0] <= Uq[0] <=rangeC[1]) and (rangeGamma[0] <= Uq[1] <=rangeGamma[1]): # check feasible space
                    fq = SFLA_SVM_CV(x_train, y_train,n1,kernel,Uq[0],Uq[1])
                    if fq < Pw[0]: # if no improvement of performance,learn from global best randomly #
                        S = random.random(num_parameter)*(frog_gb-Pw)[1:]
                        for i in range(num_parameter):
                            if S[i] > 0:
                                S[i] = min(S[i],max_step[i])
                            else:
                                S[i] = min(S[i],-max_step[i])
                        Uq = Pw[1:]+S
                        if (rangeC[0] <= Uq[0] <=rangeC[1]) and (rangeGamma[0] <= Uq[1] <=rangeGamma[1]): # check feasible space
                            fq = SFLA_SVM_CV(x_train, y_train,n1,kernel,Uq[0],Uq[1])
                            if fq < Pw[0]: # if no improvement of performance, randomly generate a new frog
                                Uq = [10**random.uniform(log10(rangeC[0]),log10(rangeC[1])),10**random.uniform(log10(rangeGamma[0]),log10(rangeGamma[1]))]
                                fq = SFLA_SVM_CV(x_train, y_train,n1,kernel,Uq[0],Uq[1])
                        else: # if not in the feasible space, randomly generate a new frog
                            Uq = [10**random.uniform(log10(rangeC[0]),log10(rangeC[1])), 10**random.uniform(log10(rangeGamma[0]),log10(rangeGamma[1]))]
                            fq = SFLA_SVM_CV(x_train, y_train,n1,kernel,Uq[0],Uq[1])            
                else: # if not in the feasible space, learn from global best randomly 
                    S = random.random(num_parameter)*(frog_gb-Pw)[1:]
                    for i in range(num_parameter):
                        if S[i] > 0:
                            S[i] = min(S[i],max_step[i])
                        else:
                            S[i] = min(S[i],-max_step[i])
                    Uq = Pw[1:]+S
                    if (rangeC[0] <= Uq[0] <=rangeC[1]) and (rangeGamma[0] <= Uq[1] <=rangeGamma[1]): # check feasible space
                        fq = SFLA_SVM_CV(x_train, y_train,n1,kernel,Uq[0],Uq[1])
                        if fq < Pw[0]: # if no improvement of performance, randomly generate a new frog
                            Uq = [10**random.uniform(log10(rangeC[0]),log10(rangeC[1])), 10**random.uniform(log10(rangeGamma[0]),log10(rangeGamma[1]))]
                            fq = SFLA_SVM_CV(x_train, y_train,n1,kernel,Uq[0],Uq[1])
                    else: # if not in the feasible space, randomly generate a new frog
                        Uq = [10**random.uniform(log10(rangeC[0]),log10(rangeC[1])), 10**random.uniform(log10(rangeGamma[0]),log10(rangeGamma[1]))]
                        fq = SFLA_SVM_CV(x_train, y_train,n1,kernel,Uq[0],Uq[1])

                #--- Upgrade the memeplex ---# 
                memeplexes[im][subindex[q-1]] = r_[fq,Uq]
                memeplexes[im] =  memeplexes[im][argsort( memeplexes[im][:,0])[::-1]]            

                i_local += 1

            im += 1
        #######--- Local search end---######    

        #--- Step 5--Shuffle memeplexes ---#
        f_parameter =  memeplexes.reshape(m*n,num_parameter+1)
        f_parameter = f_parameter[argsort(f_parameter[:,0])[::-1]]


        i_global += 1

        #--- Step 6--Check convergence ---#
        if f_parameter[0,0] > 0.99:
            print('The program was terminated because it reached the optimization goal with f = %.3f' %f_parameter[0,0])
            break
#         if   abs(frog_gb - f_parameter[0,0])<10**-4:
#             flag +=1
#         if flag > 30:
#             break
        fBest_iteration = r_[fBest_iteration,f_parameter[0,0]] 

    #######--- Global search end---######
        
    return (f_parameter[0],fBest_iteration)

In [ ]:
###--- ASVM 0.632+bootstrap---###

##--- ASVM parameter---##
start = time.process_time()
num_parameter = 2# number of parameter to optimize
num_global = 30# the maximum number of global iterations
num_local = 20# the maximum number of local iterations
m =4 # the number of memeplexes
n = 8 # the number of frogs in each memeplex
q = 5 # the number of frogs in submemeplex
n1 = 10 # number of splits for inner loop
kernel = 'rbf'
rangeC = [10**-12, 10**12] # list, float, range of parameter C,eg.[10**-2, 10**2]
rangeGamma = [10**-6, 1]

oob = BootstrapOutOfBag(n_splits=50,random_seed = 920)
Brier_split = []
for train_index, test_index in oob.split(combined_data):
    #---  Seperate traing set and test set ---#
    x_train, x_test = x.iloc[train_index][:], x.iloc[test_index][:]
    y_train = y.iloc[train_index][:]
    
    #---  Fill NaN age ---#
    x_train[isnan(x_train)] = 0
    x_test[isnan(x_test)] = 0
    
    ##---  optimize SVM with SFLA---##
    f_parameter,fBest_iteration = SFLA_RBF(num_parameter,num_global,num_local,m,n,q,n1,kernel,rangeC,rangeGamma,x_train,y_train)
    
    ##---  creat and train the model ---##
    clf = svm.SVC(kernel=kernel,C=f_parameter[1],gamma=f_parameter[2],probability=True,random_state=920)
    clf.fit(x_train, y_train)
    
    ##---  Calculate Performance ---##
    y_score_test = clf.predict_proba(x_test)[:,1]
    y_score_train = clf.predict_proba(x_train)[:,1]  
    
    # Compute error based on auc
    E_resub =  metrics.brier_score_loss(y_train, y_score_train)
    E_loop = metrics.brier_score_loss(y[test_index], y_score_test)
    
#     # Compute brier_score
#     E_resub = metrics.brier_score_loss(y_train, y_score_train)
#     E_loop = metrics.brier_score_loss(y[test_index], y_score_test)
    
    # compute no-information-rate
    y_pred = clf.predict(x)
    length = len(y_pred)
    gamma = sum(y==0)/length*(1-sum(y_pred==0)/length)+sum(y==1)/length*(1-sum(y_pred==1)/length)
    
    E_632 = 0.368*E_resub + 0.632*E_loop
    
    # compute R
    if gamma > E_resub:
        R = (E_loop - E_resub) / (gamma - E_resub)
    else:
        R = 0
        E_resub = gamma
    
    # compute weight and adjusted Score
#     weight = 0.632 / (1 - 0.368*R)
    E_632plus = E_632 + (E_resub - E_loop)*(0.368*0.632*R/(1-0.368*R))
    
#     AUC_632plus = 1 - (weight*E_loop + (1. - weight)*E_resub)

    Brier_split.append(E_632plus)
    print(Brier_split)
end = time.process_time()

print('The ASVM algorithm takes '+str(end - start)+'seconds.\n')
print('Brier Score in each split:', Brier_split)
Brier = mean(Brier_split)
print('Average of Brier Score: %.4f' % (Brier))

# Confidence interval
lower = percentile(Brier_split, 2.5)
upper = percentile(Brier_split, 97.5)
print('95%% Confidence interval: [%.4f, %.4f]' % (lower, upper))